In [1]:
# pip install gref4hsi
# pip install pykrige
# mamba install py6s

# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.scripts import coregistration
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.utils import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb


In [3]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [4]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu/2022",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs"

In [5]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-05-13 22:35:49.872379


In [6]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT a 'processed' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

In [7]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208311040_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208311435_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2022/2022-08-31-Remoy/remoy_202208310800_ntnu_hyperspectral_74m')]

In [8]:
# Establish the ancillary data paths, copy to local work space and 
from pathlib import Path
import shutil

geoid_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini')
lab_calibration_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

# Local path geoid
geoid_path = os.path.join(temp_dir, "no_kv_HREF2018A_NN2000_EUREF89.tif")
try:
    shutil.copyfile(geoid_path_minio, geoid_path)
except FileExistsError:
    pass

# Local path for configuration file
config_template_path = os.path.join(temp_dir, "config_template_path_specim.ini")
try:
    shutil.copyfile(config_template_path_minio, config_template_path)
except FileExistsError:
    pass

# Local path for lab calibration
lab_calibration_path = os.path.join(temp_dir, "lab-calibration")
try:
    shutil.copytree(lab_calibration_path_minio, lab_calibration_path)
except FileExistsError:
    pass

In [ ]:
import process_hyperspectral

# Process missions in the list
for specim_mission_folder_minio in mission_list:

    mission_name = specim_mission_folder_minio.name
    print(f"\n################\nProcessing: {mission_name}")

    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)

    # Copy to local fs if not there already
    try:
        print('Copying file')
        shutil.copytree(specim_mission_folder_minio, specim_mission_folder)
    except FileExistsError:
        pass

    # The config file is read
    config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

    # Start the specim processing (if it does not finish
    process_hyperspectral.main(str(config_yaml), 
                        str(specim_mission_folder), 
                        geoid_path, 
                        config_template_path, 
                        lab_calibration_path,
                        fast_mode = False)




################
Processing: remoy_202208311040_ntnu_hyperspectral_74m
Copying file
The file 'orthomosaic_cropped.tif' is used as as reference orthomosaic.


/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


3D model already exists and overwriting is not supported

################ Georeferencing: ################
Georeferencing file 1/7, progress is 0.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 2/7, progress is 14.285714285714286 %
Trimesh failed to find intersections for 3 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 3/7, progress is 28.571428571428573 %
Trimesh failed to find intersections for 2 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 4/7, progress is 42.857142857142854 %
Trimesh failed to find intersections for 6 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh 

/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


3D model already exists and overwriting is not supported

################ Georeferencing: ################
Georeferencing file 1/4, progress is 0.0 %
Trimesh failed to find intersections for 2 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 2/4, progress is 25.0 %
Trimesh failed to find intersections for 3 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 3/4, progress is 50.0 %
All rays were successfully intersected with trimesh
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 4/4, progress is 75.0 %
Trimesh failed to find intersections for 2 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.6

/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


3D model already exists and overwriting is not supported

################ Georeferencing: ################
Georeferencing file 1/7, progress is 0.0 %
Trimesh failed to find intersections for 4 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 2/7, progress is 14.285714285714286 %
All rays were successfully intersected with trimesh
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 3/7, progress is 28.571428571428573 %
Trimesh failed to find intersections for 1 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh translation is [2951976.50797531  293124.67589315 5627404.45909952]
Georeferencing file 4/7, progress is 42.857142857142854 %
Trimesh failed to find intersections for 3 rays
Ray tracing for these rays is retried in pyvista
All rays successfully traced in VTK
Mesh 

## 3. Convert radiance data into reflectance by division with simulated spectrum from Py6S

### 3.1 Visualize the raster data including RGB composite, data cube and ancillary

In [ ]:
# Raster and ancillary (
import rasterio
from rasterio.plot import show
import spectral as sp
import shutil

import glob

module_path = os.path.join('/home/notebook/', 'gitprojects/rad4sea/')
if module_path not in sys.path:
    sys.path.append(module_path)
import rad4sea
import rad2refl

# The radiance multiplier for the specim afx10 relative to W/(m^2*sr*nm)
radiance_multiplier = (1 / 1000) #(mW/cm^2*sr*um)*1000.0000 ->(mW/cm^2*sr*um)
radiance_multiplier *= (1e-3 / 1e-4) #(mW/cm^2*sr*um) -> (W/m^2*sr*um)
radiance_multiplier *= (1 / 1e3) # (W/m^2*sr*um) -> (W/m^2*sr*nm)



for specim_mission_folder_minio in mission_list:
    
    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    
    # Where the cube and anc data are
    cube_folder = os.path.join(specim_mission_folder, "/processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "/processed/Output/GIS/AncillaryData/")

    # Make copies of radiance data for manipulation
    for filename in os.listdir(cube_folder):
        if filename.lower().endswith(".img") or filename.lower().endswith(".hdr"):  # Check for lowercase extension
            base, ext = os.path.splitext(filename)  # Separate base name and extension

            if base.split('_')[-1] == 'reflectance':
                pass
            else:
                new_filename = f"{base}_reflectance{ext}"  # Construct new filename with suffix
                source_file = os.path.join(cube_folder, filename)
                destination_file = os.path.join(cube_folder, new_filename)
                # Copy the file
                shutil.copyfile(source_file, destination_file)

    # We only modulate the copy of the data (with _reflectance suffix)
    cube_hdr_list = glob.glob(cube_folder + '/*_reflectance.hdr')
    
    # Simulates downwelling irradiance and computes remote sensing reflectance 
    rad2refl.main(anc_folder = anc_folder, cube_list_refl = cube_hdr_list, cube_folder = cube_folder, radiance_multiplier=radiance_multiplier)



In [ ]:
import matplotlib.pyplot as plt
# Example plotting of ancillary data


refl_image_obj = sp.io.envi.open(cube_hdr_list[0])
print(cube_hdr_list[0])
plt.plot(refl_image_obj[100, 100, :].flatten())
plt.show()

refl_image_obj = sp.io.envi.open('_'.join(cube_hdr_list[0].split('_')[0:-1]) + '.hdr') 
plt.plot(refl_image_obj[100, 100, :].flatten())
plt.show()



## 4. Transfer to MinIO and Publish to GeoNode

### 4.1 Convert RGB composites into one one mosaic (no fancy stretching or similar yet)


In [ ]:
def merge_rasters(raster_file_list, output_file):
    """Merges the geotif files in a directory into one geoTIF"""
    ds_lst = list()
    for raster in raster_file_list:
        ds = gdal.Warp('', raster, format='vrt')
        ds_lst.append(ds)
    dataset = gdal.BuildVRT('', ds_lst)
    ds1 = gdal.Translate(output_file, dataset)
    del ds1  
    del dataset
    return output_file

In [ ]:
for specim_mission_folder_minio in mission_list:
    # Select a particular transect datacube:
    specim_mission_folder = os.path.join(temp_dir, specim_mission_folder_minio.name)
    
    # Where the cube and anc data are
    cube_folder = os.path.join(specim_mission_folder, "/processed/Output/GIS/HSIDatacubes/")
    anc_folder = os.path.join(specim_mission_folder, "/processed/Output/GIS/AncillaryData/")
    
    # Processed composites are now stored under
    composites_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/RGBComposites')
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')
    
    composite_list = glob.glob(composites_path+'/*.tif')
    
    # The combined version is to be deployed on GEONODE
    output_filename = os.path.join(composites_path,"combined.tif")
    
    # Then we transfer the results to MinIO
    # Processed datacubes are now stored under
    cubes_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/HSIDatacubes')
    cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

    # Processed datacubes are now stored under
    anc_cube_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/AncillaryData')
    anc_cube_path_minio = os.path.join(specim_mission_folder_minio, 'processed/ancillary')

    # Processed composites are now stored under
    composites_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/RGBComposites')
    # And should be moved to 
    composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')

    # Move datacube, composites and ancillary data to persistant storage on Minio
    # Cubes (radiance and reflectance)
    sb.storage.copy_folder(src_fold = cubes_path, dst_fold = cubes_path_minio, client=minio_client, containing_folder = False)
    
    # Composite data (One per transect and a combined for visualization)
    sb.storage.copy_folder(src_fold = composites_path, dst_fold = composites_path_minio, client = minio_client, containing_folder = False)
    
    # Ancillary data for further processing
    sb.storage.copy_folder(src_fold = anc_cube_path, dst_fold = anc_cube_path_minio, client = minio_client, containing_folder = False)


In [ ]:
# Identify datasets for publishing. Folders must contain either an ODM or Pix4D
# original orthophoto (not both) and must not contain a COG named f'{layer_name}.tif'.
# Folders must also have 'config.seabee.yaml' files where 'publish' is True
publish_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.is_publish_ready(f.parent)
    and sb.ortho.parse_config(f.parent)["publish"]
]

print("The following missions will be published to GeoNode:")
print(publish_list)

In [ ]:
# Publish
for mission_fold in publish_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    print("Preparing orthophoto for publishing.")

    # Orthophoto is termed "combined.tif"
    ortho_path = os.path.join(
        mission_fold, "processed/composites", "combined.tif"
    )

    # Standardise and save locally
    layer_name = sb.ortho.get_layer_name(mission_fold)
    temp_path = os.path.join(temp_dir, layer_name + ".tif")
    try:
        sb.geo.standardise_orthophoto(
            ortho_path,
            temp_path,
            red_band=1,
            green_band=2,
            blue_band=3,
            nodata=255,
        )
    except CalledProcessError as e:
        print(f"Failed to standardise {ortho_path}.")
        print(e)
        continue

    # Copy to MinIO and delete local version
    stan_path = os.path.join(mission_fold, "orthophoto", layer_name + ".tif")
    sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=False)
    os.remove(temp_path)

    print("Uploading to GeoServer.")

    sb.geo.upload_raster_to_geoserver(
        stan_path,
        SETTINGS.GEOSERVER_USER,
        SETTINGS.GEOSERVER_PASSWORD,
        workspace="geonode",
    )

    print("Publishing to GeoNode.")

    sb.geo.publish_to_geonode(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        workspace="geonode",
    )

    print("Updating metadata.")
    date = sb.ortho.parse_mission_data(mission_fold, parse_date=True)[2]
    abstract = sb.geo.get_html_abstract(str(mission_fold))
    metadata = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    sb.geo.update_geonode_metadata(
        layer_name,
        SETTINGS.GEONODE_USER,
        SETTINGS.GEONODE_PASSWORD,
        metadata,
    )

In [ ]:
"""Transfer the """

# Processed datacubes are now stored under
cubes_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/HSIDatacubes')
cubes_path_minio = os.path.join(specim_mission_folder_minio, 'processed/cubes')

# Processed datacubes are now stored under
anc_cube_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/AncillaryData')
anc_cube_path_minio = os.path.join(specim_mission_folder_minio, 'processed/ancillary')

# Processed composites are now stored under
composites_path = os.path.join(temp_dir, 'mission-data/processed/Output/GIS/RGBComposites')
# And should be moved to 
composites_path_minio = os.path.join(specim_mission_folder_minio, 'processed/composites')

# Move datacube, composites and ancillary data to persistant storage on Minio
#sb.storage.copy_folder(src_fold = cubes_path, dst_fold = cubes_path_minio, client=minio_client, containing_folder = False)
#sb.storage.copy_folder(src_fold = composites_path, dst_fold = composites_path_minio, client = minio_client, containing_folder = False)

# Delete the local data to make room for next mission content
#shutil.rmtree(specim_mission_folder)